In [1]:
{-# LANGUAGE TypeOperators, FlexibleContexts, TypeFamilies
 , UnicodeSyntax, ScopedTypeVariables, NoMonomorphismRestriction #-}
import Prelude ()
import Data.Manifold.TreeCover
import Data.Manifold.Shade
import Data.Random
import Data.Random.Manifold
import Data.Manifold
import Data.Manifold.Web
import Data.Manifold.DifferentialEquation
import Math.LinearMap.Category
import Data.VectorSpace
import Data.Function.Affine
import Data.Basis (Basis)
import Linear(V2(..), ex, ey, _x, _y)
import Data.Semigroup
import Data.List.NonEmpty (NonEmpty(..))
import qualified Data.Foldable as Hask
import qualified Control.Monad as Hask
import qualified Control.Comonad.Cofree as Hask
import Control.Lens
:opt no-lint
import Control.Category.Constrained.Prelude
import Control.Arrow.Constrained
test x = x `seq` putStrLn "ok"

From [diagrams](http://projects.haskell.org/diagrams):

In [2]:
import Diagrams.Prelude (p2, circle, (&), (^&), moveTo, opacity, fromVertices, Point(P))

In [3]:
type X = ℝ
type T = ℝ
type U = ℝ
type Ðx'U = ℝ
type Ðt'U = ℝ
type Ðx'Ðt'U = ℝ
type Ðx²'U = ℝ
type x × y = ℝ²
type HeatFlow = (U, Ðx'U)
et = ey :: Basis ℝ²
_t = _y :: Lens' (X × T) T
_u = _1 :: Lens' (U, Ðx'U) U
_ðx'U = _2 :: Lens' (U, Ðx'U) Ðx'U

From [dynamic-plot](http://hackage.haskell.org/package/dynamic-plot):

In [4]:
import Graphics.Dynamic.Plot.R2
import Data.Colour.Names
import Data.Colour.Manifold

colourscheme :: Shade' ℝ -> Shade (Colour ℝ)
colourscheme (Shade' u du) = interp (Shade u $ dualNorm du :: Shade ℝ)
 where Just interp = rangeOnGeodesic darkblue orange
 
prettyWebPlot :: PointsWeb ℝ² y -> DynamicPlottable
prettyWebPlot w = plot [ diagramPlot . opacity 0.5 $ fromVertices [P r₁, P r₂]
                       | ((r₁@(V2 x₁ y₁),_),(r₂@(V2 x₂ y₂),_)) <- edg ]
 where edg = webEdges w
 
colourscheme_heat :: Shade' HeatFlow -> Shade (Colour ℝ)
colourscheme_heat = f `seq` \(Shade' y e) -> f (Shade y $ dualNorm e)
 where Just (f :: Shade HeatFlow -> Shade (Colour ℝ))
          = rangeWithinVertices ((0,0), sienna)
                              [ ((-1,0), darkslateblue)
                              , ((0,1000000), seagreen) ]
test colourscheme_heat

ok

In [5]:
α = 0.01

heatEqn :: DifferentialEqn (X × T) Ðx²'U (U, Ðx'U)
heatEqn (Shade (_, (_, old_ðx'u)) rng_all) = LocalDifferentialEqn {
     _predictDerivatives =
       return . embedShade (lensEmbedding (α ∂_u/∂_t))
   , _rescanDerivatives =
       \shj shy -> ( mixShade's $ shy :|[embedShade (lensEmbedding _ðx'U)
                                         (projectShade (lensEmbedding (1 ∂_u/∂_x)) shj :: Shade' Ðx'U)]
                   , return $ projectShade (lensEmbedding (α ∂_u/∂_t)) shj )
   }

{-  (arr.LinearFunction $
        \((_u, ðx'u), ðx²'u)
       -> let ðt'u = ðx²'u * α
          in LinearMap (V2 (ðx'u, ðx²'u)
                           (ðt'u, 0)) )  -- ðt'ðx'u is unknown
    (arr.LinearFunction $
        \(LinearMap (V2 (ðx'u, ðx'ðx'u)
                        (ðt'u, _)))
       -> let ðx²'u = ðt'u / α
          in ((0, ðx'u), (ðx²'u + ðx'ðx'u)/2) ) -}

In [6]:
initState_heat :: X -> HeatFlow
initState_heat x = ( u
                   , (- 2 * s*x / cosh (s * (1 - x^2))^2 )
                   )
 where u = tanh (s * (1 - x^2))
       s = 1

plotWindow $ continFnPlot <$> [ fst . initState_heat
                              , snd . initState_heat ]

tf_heat :: Needle X -> Needle T -> PointsWeb (X × T) (Shade' HeatFlow)
tf_heat δx₀ δt₀ = fromWebNodes euclideanMetric
      $  [ (V2 x t, initState_heat x|±|[(0.01,0), (0,10)])
         | x<-[xl, xl+δx₀ .. xr], t<-[0] ]
      ++ [ (V2 x t, initState_heat x|±|[(1, 0), (0, 10)])
         | x<-[xl, xl+δx₀ .. xr], t<-tail[0, δt₀ .. 1] ]
 where (xl,xr) = (0,2)

GraphWindowSpecR2{lBound=-1.333333333333333, rBound=1.333333333333333, bBound=-1.8790981773565485, tBound=1.8790981773565432, xResolution=640, yResolution=480}

In [7]:
startSt_heat = tf_heat 0.125 0.125

-- Hask.toList startSt_heat
-- Hask.toList <$> rescanPDEOnWeb AbortOnInconsistency heatEqn startSt_heat

scrutiniseShade' :: ∀ x . x ~ HeatFlow => Shade' x -> String
scrutiniseShade' (Shade' c (Norm e)) = show c ++ "±" ++ show (arr e :: Needle x +> Needle' x)

tfs_heat = iterateFilterDEqn_static (indicateInconsistencies intersectShade's) id heatEqn startSt_heat
forM_ [Hask.toList tfs_heat ]
  $ \tfs ->
    plotWindow
       [ prettyWebPlot $ head tfs
       , plotLatest [ plot (fmap colourscheme_heat tfi)
                       & legendName ("i = "++show i)
                    | (i,tfi) <- zip [0..] tfs ] ]
Hask.mapM_ (\st -> mapM_ (\((p,y),(q,z)) -> putStrLn $
                 show p ++ ":\t" ++ prettyShowShade' y
            ++ "\n" ++ show q ++ ":\t" ++ prettyShowShade' z) (webEdges st) >> print())
    $ take 0 (Hask.toList tfs_heat)

In [8]:
import Control.Monad.Trans.Except
findErr :: (Hask.Monad m, Hask.Foldable m) => Hask.Cofree m a -> m ()
findErr (a Hask.:< q) = case Hask.toList q of
        [] -> const () <$> q
        l  -> foldr1 (>>) $ findErr<$>l

case runExcept $ findErr tfs_heat of
 Left (PropagationInconsistency pro apr) -> do
      putStrLn $ "apriori: "++show apr
      forM_ pro $ \(o,m) -> putStrLn $ show o ++ "\t-> " ++ show m
      plotWindow $ (plot apr & legendName "apriori")
                 : [plot m & legendName ("from "++show o) | (o,m)<-pro]


apriori: (-1.9,3e3)|±|[(1.8,-6e3),(0,3e3)]
V2 2.0 0.375	-> (2,-0e16)|±|[(9,0)]
V2 2.0 0.125	-> (-51,-0e4)|±|[(20,-4e4),(0,4e4)]
V2 1.875 0.25	-> (-1,-3)|±|[]
GraphWindowSpecR2{lBound=-1.333333333333333, rBound=1.333333333333333, bBound=-1.333333333333333, tBound=1.333333333333333, xResolution=640, yResolution=480}

![Initial state from which to start refining solution of the heat PDE](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/PDE-solution-filter/HeatEqn-InitialState.png)

In [9]:
length tfs_heat

27